In [16]:
from flask import Flask, render_template, request
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

Copiert und modifiziert von https://medium.com/nerd-for-tech/simple-linear-regression-with-deployment-using-flask-cafbf85fa96f

In [17]:
X = np.array([10, 20, 30, 40, 50, 10, 25, 30, 45, 50, 15, 20, 35, 40, 50, 50, 45, 30, 25, 20, 10])
Y = np.array([95, 185, 280, 370, 490, 100, 230, 290, 410, 500, 135, 200, 295, 395, 495, 480, 430, 305, 205, 175, 110])

data = pd.DataFrame({'Ads/Month':X, 'Paid/Month':Y})
data.head()

,Ads/Month,Paid/Month
0,10,95
1,20,185
2,30,280
3,40,370
4,50,490


In [18]:
model = LinearRegression().fit(X.reshape(-1, 1), Y)

In [19]:
filename = "model.sav"
joblib.dump(model, "../webserver/"+str(filename))

['../webserver/model.sav']

In [20]:
app = Flask(__name__, template_folder="../webserver")

@app.route('/')
def student():
    return render_template("home.html")


def ValuePredictor(to_predict_list):
    to_predict = np.array(to_predict_list).reshape(-1, 1)
    loaded_model = joblib.load("../webserver/model.sav")
    result = loaded_model.predict(to_predict)
    return result[0]


@app.route('/', methods=["POST", "GET"])
def result():
    if request.method == "POST":
        to_predict_list = request.form.to_dict()
        to_predict_list = list(to_predict_list.values())
        to_predict_list = list(map(float, to_predict_list))
        result = round(float(ValuePredictor(to_predict_list)), 2)
        return render_template("home.html", result=result)


if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Nov/2024 16:27:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2024 16:27:44] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2024 16:27:44] "GET /favicon.ico HTTP/1.1" 404 -
